# Overfitting

*In which we practice one thing so much that we get worse at everything else.*


### Define Constants


In [13]:
CACHE_FILE = '../cache/crabs.json'
NEXT_NOTEBOOK = '../1-models/models.ipynb'

PREDICTION_TARGET = 'Age'    # 'Age' is predicted
DATASET_COLUMNS = ['Sex','Length','Diameter','Height','Weight','Shucked Weight','Viscera Weight','Shell Weight',PREDICTION_TARGET]
REQUIRED_COLUMNS = [PREDICTION_TARGET]


### Importing Libraries


In [14]:
from notebooks.time_for_crab.mlutils import *

import numpy as np
import pandas as pd

#from sklearn.svm import SVC
#from sklearn.model_selection import cross_val_score, cross_val_predict, train_test_split
#from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

try:
    # for visual mode. `pip install -e .[visual]`
    import matplotlib.pyplot as plt
    import matplotlib
    %matplotlib inline
    import seaborn as sns
except ModuleNotFoundError:
    plt = None
    sns = None

pd.set_option('mode.copy_on_write', True)


### Load Data from Cache

In the [previous section](../0-eda/eda.ipynb), we saved the cleaned data to a cache file. Let's load it back.


In [15]:
crabs = pd.read_json(CACHE_FILE)
crabs.info()


<class 'pandas.core.frame.DataFrame'>
Index: 3893 entries, 0 to 3892
Data columns (total 11 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   Length          3893 non-null   float64
 1   Diameter        3893 non-null   float64
 2   Height          3893 non-null   float64
 3   Weight          3893 non-null   float64
 4   Shucked Weight  3893 non-null   float64
 5   Viscera Weight  3893 non-null   float64
 6   Shell Weight    3893 non-null   float64
 7   Age             3893 non-null   int64  
 8   Sex_F           3893 non-null   bool   
 9   Sex_I           3893 non-null   bool   
 10  Sex_M           3893 non-null   bool   
dtypes: bool(3), float64(7), int64(1)
memory usage: 285.1 KB


### Memory Reduction

Crabs were never known for their memory. Let's minimize the memory of our DataFrame using the smallest data types to fit the data.

The reason for this is to save computational resources and time. The smaller the data, the faster the processing.


In [16]:
crabs = data_downcasting(crabs)
crabs.info()


Memory usage of dataframe is 0.2784 MB (before)
Memory usage of dataframe is 0.0965 MB (after)
Reduced 65.3%
<class 'pandas.core.frame.DataFrame'>
Index: 3893 entries, 0 to 3892
Data columns (total 11 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   Length          3893 non-null   float16
 1   Diameter        3893 non-null   float16
 2   Height          3893 non-null   float16
 3   Weight          3893 non-null   float16
 4   Shucked Weight  3893 non-null   float16
 5   Viscera Weight  3893 non-null   float16
 6   Shell Weight    3893 non-null   float16
 7   Age             3893 non-null   int8   
 8   Sex_F           3893 non-null   bool   
 9   Sex_I           3893 non-null   bool   
 10  Sex_M           3893 non-null   bool   
dtypes: bool(3), float16(7), int8(1)
memory usage: 98.8 KB


## Overfitting Crab Age

![Large mud crab measure](https://upload.wikimedia.org/wikipedia/commons/thumb/6/65/CSIRO_ScienceImage_10696_Mud_crabs_are_caught_measured_tagged_and_released_as_part_of_the_research_into_the_effectiveness_of_green_zones_in_Moreton_Bay.jpg/1920px-CSIRO_ScienceImage_10696_Mud_crabs_are_caught_measured_tagged_and_released_as_part_of_the_research_into_the_effectiveness_of_green_zones_in_Moreton_Bay.jpg)


### Overfitting Methods

We will use the following methods to overfit the data:

1. **Train-Test Split**: Split the data into training and testing sets.
2. **Cross Validation**: Use cross-validation to train and test the model.
3. **Feature Engineering**: Create new features to overfit the model.
4. **Hyperparameter Tuning**: Tune the hyperparameters to overfit the model.
5. **Model Selection**: Select the best model to overfit the data.
7. **Regularization**: Regularize the model to prevent overfitting.
8. **Data Augmentation**: Augment the data to overfit the model.
9. **Transfer Learning**: Use transfer learning to overfit the model.
10. **Model Stacking**: Stack models to overfit the model.


### Don't Save this Data

We don't want our over-trained model to leak into the [next step](../1-models/models.ipynb).


### Onwards to Model Selection

See the [next section](../1-models/models.ipynb) for model selection.
